In [116]:
import numpy as np
import math
from scipy.optimize import minimize
import random

In [117]:
actual_file = r'\Cellulose110_8000Water_NaCL_5M.txt'
written_file = actual_file.replace(r"\C",'C')
written_file = written_file.replace('.txt','')
item_path = r'D:\all_programming\wetting-analysis\DataFiles' + actual_file

In [118]:
with open(item_path, 'r') as file:
    x_vals = {}
    y_vals = {}
    
    for row in file:
        if row.startswith('Frame: '):
            row = row.strip()
            junk, current_frame = row.split(': ')
            current_frame = int(current_frame) 
            x_vals[current_frame] = []
            y_vals[current_frame] = []

        elif row.startswith('{'):
            row = row.strip()
            row = row.replace(', ', 'ß')
            row = row.replace('} {', 'ß')
            row = row.replace('{', '')
            row = row.replace('}', '')
            broken_row = row.split('ß')

            for i in range(len(broken_row)):
                if i % 2 == 0:
                    x_vals[current_frame].append(float(broken_row[i]))
                elif i % 2 == 1:
                    y_vals[current_frame].append(float(broken_row[i]))

            x_vals_polished = {}
            for frame in x_vals.keys():
                min_x = min(x_vals[frame])
                x_vals_polished[frame] = []

                for item in x_vals[frame]:
                    x_vals_polished[frame].append(item + abs(min_x))

            x_vals = x_vals_polished

In [119]:
point_count = {}
for key in x_vals.keys():
    point_count[key] = len(x_vals[key])

In [120]:
bootstrap_iterations = 1000

rand_x_vals = {} # {0:{0:[x0,x1,...],...},...}
rand_y_vals = {}

for key in x_vals.keys():
    rand_x_vals[key] = {}
    rand_y_vals[key] = {}

    for i in range(bootstrap_iterations):
        rand_x_vals[key][i] = []
        rand_y_vals[key][i] = []

        for j in range(len(x_vals[key])):
            random_index = random.randint(0, point_count[key] - 1)
            rand_x, rand_y = x_vals[key][random_index], y_vals[key][random_index]
            rand_x_vals[key][i].append(rand_x)
            rand_y_vals[key][i].append(rand_y)


In [121]:
# Define the objective function
def objective(params, X, Y):
    a, b, y_0, C, x_0 = params
    summation = np.sum(np.abs((1/a**2)*X**2 - 2*x_0/a**2*X + (1/b**2)*Y**2 - 2*y_0/b**2*Y - C))
    return summation

In [122]:
CA_dict = {}

for frame in rand_x_vals.keys():
    CA_dict[frame] = []

    for boot_it in rand_x_vals[frame].keys():
    
        X_data = np.array(rand_x_vals[frame][boot_it])  # X values
        Y_data = np.array(rand_y_vals[frame][boot_it])  # Y values

        x_0 = max(X_data) / 2  # Generally true statement 
        x_0_pm = 5

        # Initial guess for the parameters [a, b, y_0, C, x_0]
        initial_guess = [0.4, 0.5, -11.1, 10, x_0]

        # Define bounds for the parameters: (a, b, y_0, C, x_0)
        a_B = (0.1, 100)
        b_B = (0.1, 100)
        y_0_B = (-50, -20)
        C_B = (-100, 100)
        x_0_B = (x_0 - x_0_pm, x_0 + x_0_pm)
        bounds = [a_B, b_B, y_0_B, C_B, x_0_B] 

        # Perform the optimization
        result = minimize(objective, initial_guess, args=(X_data, Y_data), method='L-BFGS-B', bounds=bounds)
        a_opt, b_opt, y_0_opt, C_opt, x_0_opt = result.x

        # Angle calculation
        x_a = x_0_opt + math.sqrt(x_0**2 + C_opt * a_opt**2)
        dydx_x_a = b_opt**2*(x_0-x_a)/(a_opt**2*(-y_0_opt))
        theta = abs(math.atan(dydx_x_a)) * 180 / math.pi
        CA_dict[frame].append(theta)  

In [123]:
# Calculate the mean and standard deviation
mean_contact_angle = np.mean(CA_dict[0])
std_contact_angle = np.std(CA_dict[0])

edit = actual_file.replace(r'\C','C')
edit = edit.replace('.txt','')
print(edit)
print(f"Mean Contact Angle: {mean_contact_angle:.2f} degrees")
print(f"Contact Angle Error: ±{std_contact_angle:.2f} degrees")

Cellulose110_8000Water_NaCL_5M
Mean Contact Angle: 51.86 degrees
Contact Angle Error: ±2.08 degrees
